In [1]:
from qa_model import answer_question
from data_manager import get_data

c:\Users\Erjon\Documents\2. HTW Berlin\3. Courses\2024 - 4\Code\thesis-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
index, questions_df, answers_df, model = get_data()

In [210]:
question = "what are some neural network architectures that can give good results on the recommeder system?"

In [4]:
new_question, similar_questions, similar_answers, answer_content = answer_question(question)

In [5]:
similar_questions

['I am sure data science as will be discussed in this forum has several synonyms or at least related fields where large data is analyzed.\n\nMy particular question is in regards to Data Mining.  I took a graduate class in Data Mining a few years back.  What are the differences between Data Science and Data Mining and in particular what more would I need to look at to become proficient in Data Mining?\n',
 'I was wondering (about a more semantic question), is there a difference between data-driven methods and machine learning? Or is it more correct to state that machine learning is a category of data-driven methods (and what then are other categories)? \n',
 "When ever the word data science pops up people generally become quick to move to machine learning. Is that the right thing? For a data scientist isn't the handling of data (collection, pre-processing, visualization, etc.,) more important?\nI am aware of the thread What is valued more in the data science job market, statistical anal

In [6]:
answer_content

'Based on the information provided in the answers, data science encompasses a broader range of activities including data manipulation, communication, and working with messy data sets, while machine learning specifically focuses on developing algorithms to learn from data and make predictions. Data science involves applying mathematical and computer science algorithms to extract useful information from disordered data, while machine learning aims to find the best parameters for an equation to predict target values accurately. Therefore, data science is a multidisciplinary field that includes machine learning as one of its components.'

In [8]:
def perturb_question(question):
    words = question.split()
    perturbed_questions = []
    for i in range(len(words)):
        new_question = words[:i] + words[i+1:]
        perturbed_questions.append(' '.join(new_question))
    return perturbed_questions

# Original question
original_question = "How does the COVID-19 vaccine work?"

In [9]:
perturb_question(original_question)

['does the COVID-19 vaccine work?',
 'How the COVID-19 vaccine work?',
 'How does COVID-19 vaccine work?',
 'How does the vaccine work?',
 'How does the COVID-19 work?',
 'How does the COVID-19 vaccine']

In [16]:
questions_df_test = questions_df.sample(10)

In [182]:
import faiss
import numpy as np
import ast


def convert_embedding(str_embedding):
    # Remove brackets and split the string by spaces
    str_embedding = str_embedding.strip("[]")
    # Split the string into a list of strings, each representing a float
    list_of_strings = str_embedding.split()
    # Convert each string in the list to a float and create a numpy array
    return np.array([float(x) for x in list_of_strings], dtype=np.float32)

embeddings_matrix = np.array([convert_embedding(e) for e in questions_df['embedding'].astype(str).values])


In [191]:
from faiss import normalize_L2

d = embeddings_matrix.shape[1]
index = faiss.IndexFlatIP(d)
normalize_L2(embeddings_matrix)
index.add(embeddings_matrix)

index_selection = 'cos'

In [194]:
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

index = faiss.read_index("data/cosine_index.faiss")
questions_df = pd.read_csv('data/questions.csv', index_col='Id')
answers_df = pd.read_csv('data/answers.csv', index_col='Id')
model = SentenceTransformer('all-MiniLM-L6-v2')

In [211]:
def answer_question1(new_question, n=5, index=index, questions=questions_df, answers=answers_df, model=model):
    # Encode the question to get its embedding, ensuring it's in the correct shape
    

    # Encode the question to get its embedding, ensuring it's in the correct shape
    question_embedding = model.encode([new_question]).reshape(1, -1)
    #faiss.normalize_L2(question_embedding.reshape(1, -1))
    # Search the index for the n closest questions by inner product
    D, I = index.search(question_embedding, n)        
    # get the similarity score of the closest question
    question_ids = questions.iloc[I[0]].index
    distance_normalized = D
    
    # Prepare lists to hold the similar questions and their answers
    similar_questions = []
    similar_answers = pd.DataFrame()
    
    # Loop through each similar question ID to get the question text and its answers
    for q_id in question_ids:
        similar_questions.append(questions.loc[q_id].Body)  # Assuming there's a QuestionText column
        q_answers = answers[answers['ParentId'] == q_id]
        similar_answers = pd.concat([similar_answers, q_answers], ignore_index=True)

    
    content = f"I have a question that I need answered based solely on the information contained in the following 5 answers. Please do not use any external knowledge or pre-existing information you might have. Just use the details provided in these answers to formulate the best possible response. If the answer is not in the provided information, answer that you dont have enough information to provide an answer. Be coincise.\n\n\n"


    for i, answer in enumerate(similar_answers['Body']): # Assumes a list of answers
        content += f"Start of Answer {i+1}: {answer}\n End of Answer {i+1}\n\n\n"      
    content += f"Question: {new_question}"
    client = OpenAI(api_key=chatgpt_key)
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "assistant",
                "content": content,
            }
        ],
        model="gpt-3.5-turbo",
        temperature=0.0,
        max_tokens=150,
    )

    answer_content = chat_completion.choices[0].message.content

    return new_question, similar_questions, similar_answers, answer_content, distance_normalized

In [212]:
new_question, similar_questions, similar_answers, answer_content, similarity = answer_question1(question)

In [206]:
new_question

'what are some neural network architectures that can give good results on the recommeder system?'

In [213]:
similar_questions

["When I first read about neural networks, I learned that Backpropagation is the algorithm used to train the neural network. I am interested if there are other alternatives (or better?) to BP. \n\nWhat are the other training algorithms used in NN? And is BP is the best one, and that's why almost everyone uses it for training the NN model?\n",
 'What are the best machine learning techniques to classify responders to a medicine if I have:\n\n\nClinical data with ~200 features (age, education, marital status etc.)\nGene data with around 250K features (genom data (snips) taken from the patient (DNA analysis))\nNumber of obs. ~ 4K (the data is taken from a study on 4000 patients).\n\n\nPlease advise.\n',
 "Disclaimer: I am almost a complete novice when it comes to tensorflow, keras, coding in general, and neural networks/data science.\nWhile reading papers on novel architectures for neural nets, I see diagrams and such describing their ideas, and then they present their results, with no cod

In [208]:
answer_content

'Based on the information provided in the answers, some neural network architectures that can potentially give good results on a recommender system include feedback-alignment (FA), Direct Feedback Alignment (DFA), Indirect Feedback Alignment (IFA), custom defined layers in Keras/TensorFlow, one-dimensional convolutions using tf.nn.conv1d in TensorFlow, and RNN networks like LSTMs and GRUs. However, the specific details on how these architectures are implemented and their effectiveness in recommender systems are not explicitly mentioned in the provided information.'

In [209]:
similarity

array([[0.56415224, 0.5240064 , 0.5211463 , 0.5211413 , 0.51799667]],
      dtype=float32)